<a href="https://colab.research.google.com/github/datasensum-fabio/Coursera_Capstone/blob/master/FOURSQUARE_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import requests 
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Imprt data:
- List all towns in Italy by population
- list all towns in Italy by Taxable Income

**Source: ISTAT**

In [74]:
#df with population per town
df_pop = pd.read_csv('/content/drive/MyDrive/Datasets/ISTAT/DCIS_POPRES1_08122020222714558.csv')

#df with Taxable income per town
df_Reddito = pd.read_csv('/content/drive/MyDrive/Datasets/ISTAT/MEF_REDDITIIRPEF_COM_07122020071748864.csv')

#df with Region and Province per town
df_denominazione= pd.read_csv('/content/drive/MyDrive/Datasets/ISTAT/Codici-statistici-e-denominazioni-al-01_07_2020.csv',encoding = "ISO-8859-1",sep=';')
df_denominazione = df_denominazione[['Denominazione (Italiana e straniera)','Codice Comune formato alfanumerico','Sigla automobilistica','Denominazione Regione']]

#df with Altitude per each town
df_altitude = pd.read_csv('/content/drive/MyDrive/Datasets/ISTAT/Elab_Altimetrie_DEM.csv',encoding = "ISO-8859-1",sep=';')
df_altitude = df_altitude[['NOME','MEDIA','STD','PRO_COM']]

# selecting only Taxable Income column
df_Reddito = df_Reddito[df_Reddito.TIPO_DATO_MEF=='TAXABINCR']
df_Reddito = df_Reddito[['Territorio','Value']]

df_pop= df_pop[['Territorio','Value']][1:]#.Territorio.value_counts().head(20)

#selecting towns with more than 10.000 people
df_pop = df_pop[df_pop.Value>10000]


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (7,8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [75]:
#checking if there are town with same name
len(df_pop.value_counts()) == len(df_pop)


True

In [82]:
#mergind Income and Population tables
df = pd.merge(df_pop,df_Reddito,how='inner',on='Territorio')
df.columns = ['Town','Population','Taxable Income']
#Calculating TARGET VAIABLE
df['Taxable_Income_Pro_capite'] = df['Taxable Income']/df['Population']
df.sort_values('Taxable_Income_Pro_capite',ascending=False)

#Merging Regione and Province per town
df = pd.merge(df,df_denominazione,how='left',left_on='Town',right_on='Denominazione (Italiana e straniera)')
#Checking if merge working ok
df['Denominazione (Italiana e straniera)'].isnull().sum()

0

In [83]:
df.drop('Denominazione (Italiana e straniera)',inplace=True,axis=1)


In [84]:
cols = list(df.columns)
cols[-2] = 'Prov'
cols[-1] = 'Regione'
df.columns = cols
df.head()

,Town,Population,Taxable Income,Taxable_Income_Pro_capite,Codice Comune formato alfanumerico,Prov,Regione
0,Torino,870952,1.498548e+10,17205.862875,1272,TO,Piemonte
1,Vercelli,46558,7.421240e+08,15939.773680,2158,VC,Piemonte
2,Verona,259608,4.653083e+09,17923.496029,23091,VR,Veneto
3,Vicenza,111764,1.920181e+09,17180.677696,24116,VI,Veneto
4,Belluno,35872,6.657425e+08,18558.833854,25006,BL,Veneto


In [85]:
#merging with df_altitude
df = pd.merge(df,df_altitude,how='left',left_on='Codice Comune formato alfanumerico',right_on='PRO_COM').drop(['NOME','PRO_COM','Codice Comune formato alfanumerico'],axis=1)
df.head()

,Town,Population,Taxable Income,Taxable_Income_Pro_capite,Prov,Regione,MEDIA,STD
0,Torino,870952,1.498548e+10,17205.862875,TO,Piemonte,"266,4043","70,81829"
1,Vercelli,46558,7.421240e+08,15939.773680,VC,Piemonte,"128,3015","6,791454"
2,Verona,259608,4.653083e+09,17923.496029,VR,Veneto,"131,6091","124,5421"
3,Vicenza,111764,1.920181e+09,17180.677696,VI,Veneto,"39,25313","14,27084"
4,Belluno,35872,6.657425e+08,18558.833854,BL,Veneto,"781,9634","440,4044"


In [86]:
cols = list(df.columns)
cols[-2] = 'AVG_Altitude'
cols[-1] = 'STD_Altitude'
df.columns = cols
df.head()

,Town,Population,Taxable Income,Taxable_Income_Pro_capite,Prov,Regione,AVG_Altitude,STD_Altitude
0,Torino,870952,1.498548e+10,17205.862875,TO,Piemonte,"266,4043","70,81829"
1,Vercelli,46558,7.421240e+08,15939.773680,VC,Piemonte,"128,3015","6,791454"
2,Verona,259608,4.653083e+09,17923.496029,VR,Veneto,"131,6091","124,5421"
3,Vicenza,111764,1.920181e+09,17180.677696,VI,Veneto,"39,25313","14,27084"
4,Belluno,35872,6.657425e+08,18558.833854,BL,Veneto,"781,9634","440,4044"


In [88]:

df.AVG_Altitude.fillna(0,inplace=True)
df.STD_Altitude.fillna(0,inplace=True)
df['AVG_Altitude'] = df.apply(lambda x: int(str(x['AVG_Altitude']).split(',')[0]),axis=1)
df['STD_Altitude'] = df.apply(lambda x: int(str(x['STD_Altitude']).split(',')[0]),axis=1)

In [90]:
df.Prov.fillna('NA',inplace=True)


#API to get Latitude and Longitude for all cities
## **LOCATION IQ**

In [48]:
df1 = df
df1.head()

,Town,Population,Taxable Income,Taxable_Income_Pro_capite,Prov,Regione
0,Torino,870952,1.498548e+10,17205.862875,TO,Piemonte
1,Vercelli,46558,7.421240e+08,15939.773680,VC,Piemonte
2,Verona,259608,4.653083e+09,17923.496029,VR,Veneto
3,Vicenza,111764,1.920181e+09,17180.677696,VI,Veneto
4,Belluno,35872,6.657425e+08,18558.833854,BL,Veneto


In [84]:
df = df1
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1213 entries, 0 to 1212
Data columns (total 6 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Town                       1213 non-null   object 
 1   Population                 1213 non-null   int64  
 2   Taxable Income             1213 non-null   float64
 3   Taxable_Income_Pro_capite  1213 non-null   float64
 4   Prov                       1150 non-null   object 
 5   Regione                    1213 non-null   object 
dtypes: float64(2), int64(1), object(3)
memory usage: 66.3+ KB


In [86]:
df['Lat'] = 999.0 #assign arbitrary high number
df['Long'] = 999.0 #assign arbitrary high number
df.index = df.Town




In [ ]:
token = 'pk.39e08a4d52c7a6c0a680c4ac7353a0a6'
latitude=[]
longitude=[]
towns =[]
countlon =0
countlat =0

#loop to avoid API issues (many time it doesn't work at first time)

while len(df[(df['Lat']>=998)|(df['Long']>=998)].Town)>=1:
  print('Cities mapped {}'.format(len(df[~(df['Lat']>=998)|(df['Long']>=998)].Town)))

#loop to retrieve Lat e Long to all towns where the value is wrong
  for x in df[(df['Lat']>=998)|(df['Long']>=998)].Town:

    lat=0
    lon=0

    query = x +', '+ df.loc[x]['Prov'] + ', '+df.loc[x]['Regione'] +', Italy'
    url = 'https://us1.locationiq.com/v1/search.php?key={}&q={}&limit=1&countrycodes=IT&format=json'.format(token,query)
    results = requests.get(url).json()

    try:
      lat = results[0]['lat']
      df.at[x,'Lat'] = lat
      countlat= countlat+1
    except:
      print('Towns mapped: ' + str(countlat+260))

    try:
      lon = results[0]['lon']
      df.at[x,'Long'] = lon
      countlon= countlon+1
    except:
      print('error: '+x)
  

In [110]:
df

,Town,Population,Taxable Income,Taxable_Income_Pro_capite,Prov,Regione,Lat,Long
Town,,,,,,,,
Torino,Torino,870952,1.498548e+10,17205.862875,TO,Piemonte,45.067755,7.682489
Vercelli,Vercelli,46558,7.421240e+08,15939.773680,VC,Piemonte,45.325156,8.422767
Verona,Verona,259608,4.653083e+09,17923.496029,VR,Veneto,45.438496,10.992412
Vicenza,Vicenza,111764,1.920181e+09,17180.677696,VI,Veneto,45.548831,11.547883
Belluno,Belluno,35872,6.657425e+08,18558.833854,BL,Veneto,46.139738,12.219297
...,...,...,...,...,...,...,...,...
Scarperia e San Piero,Scarperia e San Piero,12206,1.845611e+08,15120.521383,FI,Toscana,44.010965,11.331349
Casciana Terme Lari,Casciana Terme Lari,12287,1.595215e+08,12982.950842,PI,Toscana,43.552159,10.594207
Montoro,Montoro,19620,1.581410e+08,8060.192100,AV,Campania,40.832876,14.791776


In [126]:
#Adjusting Towns with wrong data
df[(df['Lat']>=46.7)|(df['Long']>=18.3)|(df['Lat']<=36.8)|(df['Long']<=7.4)]






,Town,Population,Taxable Income,Taxable_Income_Pro_capite,Prov,Regione,Lat,Long
Town,,,,,,,,
Avigliana,Avigliana,12599,204195577.0,16207.284467,TO,Piemonte,45.077145,7.398616
Aosta,Aosta,34052,582619938.0,17109.712733,AO,Valle d'Aosta/Vallée d'Aoste,45.737088,7.319665
Brunico/Bruneck,Brunico/Bruneck,16774,362963066.0,21638.432455,BZ,Trentino-Alto Adige/Südtirol,46.794706,11.929792
Bressanone/Brixen,Bressanone/Brixen,22572,462290165.0,20480.691343,BZ,Trentino-Alto Adige/Südtirol,46.703591,11.647639
Giaveno,Giaveno,16425,249605905.0,15196.706545,TO,Piemonte,45.042005,7.351994
Ispica,Ispica,16334,137922053.0,8443.862679,RG,Sicilia,36.785832,14.907637
Melendugno,Melendugno,10001,96730295.0,9672.062294,LE,Puglia,40.271701,18.336911
Pachino,Pachino,22312,150363425.0,6739.128048,SR,Sicilia,36.715209,15.091477
Pinerolo,Pinerolo,36178,574361544.0,15875.989386,TO,Piemonte,44.887309,7.331945


In [ ]:
df= pd.read_csv('/content/drive/MyDrive/Datasets/ISTAT/201209_Cities_Lat_Long_Italy.csv')
df.index = df.Town
df.head()

In [332]:
+15/111

0.13513513513513514

In [338]:
#population in a radius 0.13degree (15km)

def calc_pop_radius(Lat_Long,radisKM):

  lat = Lat_Long[0]
  lng = Lat_Long[1]
  radio = radisKM/111

  result = df[((df['Lat']>lat-radio)&(df['Lat']<lat+radio))&
    ((df['Long']>lng-radio)&(df['Long']<lng+radio))]['Population'].sum()
  return result

calc_pop_radius([38,15],15)

df['Pop_rad_15km'] = df.apply(lambda x: calc_pop_radius(x[['Lat','Long']],15),axis=1)

df.head()

,Town,Town.1,Town.1.1,Population,Taxable Income,Taxable_Income_Pro_capite,Prov,Regione,Lat,Long,4$_Restaurant,4$_Restaurant_Pro_capite,Pop_rad_15km
Town,,,,,,,,,,,,,
Torino,Torino,Torino,Torino,870952,1.498548e+10,17205.862875,TO,Piemonte,45.067755,7.682489,38,0.000044,1274810
Vercelli,Vercelli,Vercelli,Vercelli,46558,7.421240e+08,15939.773680,VC,Piemonte,45.325156,8.422767,8,0.000172,46558
Verona,Verona,Verona,Verona,259608,4.653083e+09,17923.496029,VR,Veneto,45.438496,10.992412,44,0.000169,381353
Vicenza,Vicenza,Vicenza,Vicenza,111764,1.920181e+09,17180.677696,VI,Veneto,45.548831,11.547883,55,0.000492,182352
Belluno,Belluno,Belluno,Belluno,35872,6.657425e+08,18558.833854,BL,Veneto,46.139738,12.219297,19,0.000530,46076


# **API FOURSQUARE**

getting venues for all towns in italy

In [257]:
df = pd.read_csv('/content/drive/MyDrive/Datasets/ISTAT/201209_Cities_Lat_Long_Italy.csv')

CLIENT_ID = 'H00GRYP5I4XZXSQXQ5GLCBNFHOGXX1LWRRUQOPBANTAVG0R0' # your Foursquare ID
CLIENT_SECRET = '1YY5T4ZWLEAWQPNGWN2QIEK1RROH2UWKHZKZLLT4UIPH2ET4' # your Foursquare Secret
ACCESS_TOKEN = 'DTGRP141XRIJKZI031RR4Y231NMGZT2AAQXK02SIUYZMDVYF' # your FourSquare Access Token
VERSION = '20180604'



In [264]:
#inzializing Column with count of $$$$ restaurant with dummy var
df['4$_Restaurant'] =-1

Getting Venues with High Price from FOURSQUARE

In [357]:
df.index = df.Town



radius = 15000
LIMIT = 500
price ='3,4'
i=0
stampa=[]

for x in df[df['4$_Restaurant']==0]['Town']:

  lat = df.loc[x]['Lat']
  lng = df.loc[x]['Long']

  url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&price={}&radius={}&limit={}'.format(
      CLIENT_ID, 
      CLIENT_SECRET, 
      VERSION,
      lat, 
      lng,
      price,
      #categoryId, 
      radius, 
      LIMIT)
  
  results = requests.get(url).json()["response"]#['groups'][0]['items']
  df.at[x,'4$_Restaurant'] = results['totalResults']
  i=i+1
  stampa.append(x + ': {}'.format(results['totalResults']))

  if i == 5:
    i=0
    print(stampa)
    stampa=[]


['Monza: 50', 'Canegrate: 54', 'Casoria: 34', 'Formia: 17', 'Laterza: 0']


In [133]:
df.to_csv('/content/drive/MyDrive/Datasets/ISTAT/201209_Cities_Lat_Long_Italy.csv')

In [429]:
# function to extract cat Name
def get_cat_name(x):
  try:
    return x[0]['shortName']
  except:
    return None
#list need as check for loop in following cell    
list_categorie =[]
df['Tot_Venues'] = -1
#inizialization with dummy var

In [ ]:
#Search all venues in radius of 15km
i=3
stampa=[]
radius = 15000
LIMIT = 500
df.index=df.Town

#First loop go trough all towns
for y in df[~(df['Tot_Venues'] > -1)]['Town']:
  lat= df['Lat'].loc[y]
  lng = df['Long'].loc[y]  
  url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
      CLIENT_ID, 
      CLIENT_SECRET, 
      VERSION,
      lat, 
      lng,
      #price,
      #categoryId, 
      radius, 
      LIMIT)
  results = requests.get(url).json()["response"]#['groups'][0]['items']
  try:
    df_venues =pd.DataFrame(results['venues'])[['id','name','categories']]
    df.at[y,'Tot_Venues'] = len(df_venues)
    df_venues['cat_name'] = df_venues.apply(lambda x: get_cat_name(x['categories']),axis=1)
    
    #2nd loop for all categories found 
    for x in df_venues.cat_name.value_counts().index:
      
      #if we have already encountered the category
      if x+'_V' in list_categorie:
        df.at[y,x+'_V'] = df_venues.cat_name.value_counts()[x]
      #case it is the first time we foud this category we need to add a new columns
      else:
        #we add a colum with cat name and dummy var
        df[x+'_V'] = 0
        list_categorie.append(x+'_V')
        df.at[y,x+'_V'] = df_venues.cat_name.value_counts()[x]
        print(x+': Added')

  except:
    df[y,'Tot_Venues'] = 0
    print(y + ': No Venues Found') 

  i=i+1
  stampa.append(y)
  if i == 5:
    i=0
    print(stampa)
    stampa=[]



In [110]:
#drop columns with venues that are very rare <15

col_to_drop =pd.Series(df.iloc[:,15:].sum()<15)
col_to_drop = list(col_to_drop[col_to_drop==True].index)
df.drop(col_to_drop,axis=1,inplace=True)

In [111]:
df.shape

(1213, 421)

MACHINE LEARNING MODELS

In [134]:
lisa

['Town',
 'Population',
 'Taxable Income',
 'Taxable_Income_Pro_capite',
 'Prov',
 'Regione',
 'Lat',
 'Long',
 '4$_Restaurant',
 '4$_Restaurant_Pro_capite',
 'Pop_rad_15km',
 '4$_Restaurant_Pro_capite_2',
 'Italian_V',
 'Office_V',
 'Café_V',
 'Tot_Venues',
 'AVG_Altitude',
 'STD_Altitude']

In [135]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

rf = RandomForestRegressor(random_state=0)
y=df['Taxable_Income_Pro_capite']
X = df[['Lat','Long']]
cv_scores = cross_val_score(rf, X, y,cv=10)
print('Only Lat and Lng - Mean cross-validation score (10-fold): {:.3f}'.format(np.mean(cv_scores)))


rf = RandomForestRegressor(random_state=0)
X = df[['Population','Lat','Long','4$_Restaurant','4$_Restaurant_Pro_capite','Pop_rad_15km','Tot_Venues','AVG_Altitude',
        'STD_Altitude','4$_Restaurant_Pro_capite_2']]
cv_scores = cross_val_score(rf, X, y,cv=10)
print('All Macro Variables - Mean cross-validation score (10-fold): {:.3f}'.format(np.mean(cv_scores)))


rf = RandomForestRegressor(random_state=0)
X = df[['Population','4$_Restaurant','4$_Restaurant_Pro_capite','Pop_rad_15km','Tot_Venues','AVG_Altitude',
        'STD_Altitude','4$_Restaurant_Pro_capite_2']]
cv_scores = cross_val_score(rf, X, y,cv=10)
print('All Macro Variables (ex Lat, Long) - Mean cross-validation score (10-fold): {:.3f}'.format(np.mean(cv_scores)))






Only Lat and Lng - Mean cross-validation score (10-fold): 0.804
All Macro Variables - Mean cross-validation score (10-fold): 0.818
All Macro Variables (ex Lat, Long) - Mean cross-validation score (10-fold): 0.508


In [140]:
X = df.drop(['Town','Taxable Income','Taxable_Income_Pro_capite', 'Prov', 'Regione'],axis=1)
y=df['Taxable_Income_Pro_capite']
rf.fit(X,y)
imp =pd.Series(rf.feature_importances_,index=X.columns)

print('TOP 10 Features')
print(pd.Series(rf.feature_importances_,index=X.columns).sort_values(ascending=False)[:10])
print('BOTTOM 10 Features')
print(pd.Series(rf.feature_importances_,index=X.columns).sort_values(ascending=False)[-10:])

Lat                           0.759458
Pop_rad_15km                  0.030258
Long                          0.024461
Population                    0.021762
4$_Restaurant_Pro_capite_2    0.010716
4$_Restaurant                 0.006089
4$_Restaurant_Pro_capite      0.005613
STD_Altitude                  0.005161
AVG_Altitude                  0.004910
Tot_Venues                    0.003610
dtype: float64
Other Repair_V           0.000005
Tennis_V                 0.000005
Exhibit_V                0.000005
College & Education_V    0.000004
Mattress Store_V         0.000003
German_V                 0.000002
Gluten-free_V            0.000001
Insurance Office_V       0.000001
Bagels_V                 0.000001
Fishing Store_V          0.000001
dtype: float64


In [142]:
rf = RandomForestRegressor(random_state=0)
y=df['Taxable_Income_Pro_capite']
X = df.drop(['Lat','Long','Town','Taxable Income','Taxable_Income_Pro_capite', 'Prov', 'Regione'],axis=1)
cv_scores = cross_val_score(rf, X, y,cv=10)
print('All Variables (ex Lat, Long) Mean cross-validation score (10-fold): {:.3f}'.format(np.mean(cv_scores)))

rf = RandomForestRegressor(random_state=0)
X = df.drop(['Lat','Long','Town','Taxable Income','Taxable_Income_Pro_capite', 'Prov', 'Regione',],axis=1)
X = X.drop(list(imp[100:].index),axis=1)
cv_scores = cross_val_score(rf, X, y,cv=10)
print('Top 100 Variables (ex Lat, Long) Mean cross-validation score (10-fold): {:.3f}'.format(np.mean(cv_scores)))

rf = RandomForestRegressor(random_state=0)
X = df.drop(['Lat','Long','Town','Taxable Income','Taxable_Income_Pro_capite', 'Prov', 'Regione',],axis=1)
X = X.drop(list(imp[300:].index),axis=1)
cv_scores = cross_val_score(rf, X, y,cv=10)
print('Top 300 Variables (ex Lat, Long) Mean cross-validation score (10-fold): {:.3f}'.format(np.mean(cv_scores)))


All Variables (ex Lat, Long) Mean cross-validation score (10-fold): 0.567
Top 100 Variables (ex Lat, Long) Mean cross-validation score (10-fold): 0.529
Top 300 Variables (ex Lat, Long) Mean cross-validation score (10-fold): 0.553


In [143]:
rf = RandomForestRegressor(random_state=0)
X = df.drop(['Lat','Long','Town','Taxable Income','Taxable_Income_Pro_capite', 'Prov', 'Regione',],axis=1)
X = X.drop(list(imp[400:].index),axis=1)
cv_scores = cross_val_score(rf, X, y,cv=10)
print('Top 400 Variables (ex Lat, Long) Mean cross-validation score (10-fold): {:.3f}'.format(np.mean(cv_scores)))

Top 400 Variables (ex Lat, Long) Mean cross-validation score (10-fold): 0.549


# K NeighborsRegressor

In [5]:
df = pd.read_csv('/content/drive/MyDrive/Datasets/ISTAT/201209_Cities_Lat_Long_Italy.csv')
df.drop('Unnamed: 0',axis=1,inplace=True)
df.Prov.fillna('NA',inplace=True)

In [6]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingRegressor



In [17]:
df.sort_values('Town',ascending=True,inplace=True)
df_Validation =df[:113]
df_Train_Test = df[113:]

df_Validation

,Town,Population,Taxable Income,Taxable_Income_Pro_capite,Prov,Regione,Lat,Long,4$_Restaurant,4$_Restaurant_Pro_capite,Pop_rad_15km,4$_Restaurant_Pro_capite_2,Italian_V,Office_V,Café_V,Landmark_V,Art Gallery_V,Tot_Venues,Boutique_V,Bank_V,Gourmet_V,Apparel_V,Church_V,Ice Cream_V,Pharmacy_V,Jewelry_V,Men's Store_V,Salon / Barbershop_V,Women's Store_V,Doctor's Office_V,Accessories_V,Plaza_V,Building_V,Optical_V,Historic Site_V,Food & Drink_V,Convention Center_V,Sandwiches_V,School_V,Adult Boutique_V,...,African_V,Light Rail_V,Skate Park_V,Track_V,Preschool_V,Comedy Club_V,Home Services_V,Bathing Area_V,Stables_V,College & Education_V,Corporate Amenity_V,ATM_V,Baseball Field_V,Hot Spring_V,Preserve_V,Alternative Healer_V,Recruiting Agency_V,Sports Club_V,Shipping Store_V,Strip Club_V,Liquor Store_V,Hot Dogs_V,Surf Spot_V,Volcano_V,Hotel Bar_V,Airport_V,Brazilian_V,Daycare_V,Billiards_V,Whisky Bar_V,Food Stand_V,Football_V,Medical School_V,Golf Course_V,Skating Rink_V,Fishing Store_V,Circus_V,Auto Workshop_V,AVG_Altitude,STD_Altitude
140,Abano Terme,20274.0,3.358855e+08,16567.300829,PD,Veneto,45.360314,11.789783,66.0,0.003255,389502.0,0.000169,0,6,10,0,1,124,1,2,0,0,0,1,0,1,0,3,1,1,0,3,2,0,1,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,3
141,Abbiategrasso,32855.0,5.309227e+08,16159.570689,MI,Lombardia,45.398733,8.916230,51.0,0.001552,253434.0,0.000201,3,6,7,0,0,125,0,5,0,1,2,1,2,1,0,4,0,4,1,4,1,1,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,102,12
142,Acate,11486.0,7.020698e+07,6112.395873,RG,Sicilia,37.025040,14.492981,3.0,0.000261,132607.0,0.000023,2,1,7,0,0,125,1,0,0,1,0,1,2,0,1,4,1,0,0,5,0,0,2,0,0,2,1,0,...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,107,68
143,Acerra,59525.0,4.443056e+08,7464.184259,NA,Campania,40.946053,14.375008,35.0,0.000588,2046067.0,0.000017,0,3,3,0,0,115,1,1,0,0,2,0,4,0,1,7,1,2,0,1,1,0,0,2,0,0,6,0,...,0,2,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,27,3
144,Aci Castello,18310.0,2.512074e+08,13719.682632,CT,Sicilia,37.554211,15.148560,22.0,0.001202,568072.0,0.000039,2,0,4,0,0,99,0,0,0,0,1,3,0,1,0,1,0,2,0,1,2,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,81,63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282,Belpasso,28072.0,2.070033e+08,7374.014392,CT,Sicilia,37.589187,14.978239,24.0,0.000855,612205.0,0.000039,0,1,7,0,0,104,0,1,0,1,3,0,1,1,0,3,0,3,0,2,3,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,349,448
53,Benevento,58794.0,7.104545e+08,12083.793380,BN,Campania,41.129559,14.782464,6.0,0.000102,58794.0,0.000102,6,1,9,0,0,125,0,0,0,1,1,2,0,1,0,1,0,1,0,5,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,225,87
110,Bergamo,121781.0,2.470276e+09,20284.575311,BG,Lombardia,45.756656,9.754219,30.0,0.000246,203457.0,0.000147,3,0,6,0,0,121,0,5,0,2,1,2,1,0,0,1,0,1,0,0,2,0,0,0,0,0,1,0,...,0,3,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,271,56
198,Bernalda,12115.0,1.076349e+08,8884.431449,MT,Basilicata,40.412818,16.691990,4.0,0.000330,12115.0,0.000330,4,2,6,0,0,121,0,2,0,0,3,1,1,0,0,0,1,1,0,3,2,0,0,1,0,4,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,47,49


In [25]:
scaler = MinMaxScaler()
knn = KNeighborsRegressor()

In [26]:

X = df.drop(['Town','Taxable Income','Taxable_Income_Pro_capite', 'Prov', 'Regione'],axis=1)
y=df['Taxable_Income_Pro_capite']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
knn.fit(X_train_scaled,y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                    weights='uniform')

In [38]:
results=[]
for k in np.arange(1,50):
  knn = KNeighborsRegressor(n_neighbors=k)
  scores=[]
  for x in np.arange(4):

    X_train, X_test, y_train, y_test = train_test_split(X, y)

    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    knn.fit(X_train_scaled,y_train)
    
    scores.append(knn.score(X_test_scaled,y_test))

  results.append(np.mean(scores))


  #knn.score(X_test_scaled,y_test)
  





In [65]:

gb = GradientBoostingRegressor(random_state=0)
y=df['Taxable_Income_Pro_capite']
X = df[['Lat','Long']]
cv_scores = cross_val_score(gb, X, y,cv=10)
print('Only Lat and Lng - Mean cross-validation score (10-fold): {:.3f}'.format(np.mean(cv_scores)))


gb = GradientBoostingRegressor(random_state=0)
X = df[['Population','Lat','Long','4$_Restaurant','4$_Restaurant_Pro_capite','Pop_rad_15km','Tot_Venues','AVG_Altitude',
        'STD_Altitude','4$_Restaurant_Pro_capite_2']]
cv_scores = cross_val_score(gb, X, y,cv=10)
print('All Macro Variables - Mean cross-validation score (10-fold): {:.3f}'.format(np.mean(cv_scores)))


gb = GradientBoostingRegressor(random_state=0)
X = df[['Population','4$_Restaurant','4$_Restaurant_Pro_capite','Pop_rad_15km','Tot_Venues','AVG_Altitude',
        'STD_Altitude','4$_Restaurant_Pro_capite_2']]
cv_scores = cross_val_score(gb, X, y,cv=10)
print('All Macro Variables (ex Lat, Long) - Mean cross-validation score (10-fold): {:.3f}'.format(np.mean(cv_scores)))

gb = GradientBoostingRegressor(random_state=0)
y=df['Taxable_Income_Pro_capite']
X = df.drop(['Town','Taxable Income','Taxable_Income_Pro_capite', 'Prov', 'Regione'],axis=1)
cv_scores = cross_val_score(gb, X, y,cv=10)
print('All Variables (incl. Lat, Long) Mean cross-validation score (10-fold): {:.3f}'.format(np.mean(cv_scores)))

Only Lat and Lng - Mean cross-validation score (10-fold): 0.809
All Macro Variables - Mean cross-validation score (10-fold): 0.828
All Macro Variables (ex Lat, Long) - Mean cross-validation score (10-fold): 0.574
All Variables (incl. Lat, Long) Mean cross-validation score (10-fold): 0.826


In [48]:
X = df.drop(['Town','Taxable Income','Taxable_Income_Pro_capite', 'Prov', 'Regione'],axis=1)
y=df['Taxable_Income_Pro_capite']
gb.fit(X,y)
imp =pd.Series(gb.feature_importances_,index=X.columns)

print('TOP 50 Features')
print(pd.Series(gb.feature_importances_,index=X.columns).sort_values(ascending=False)[:50])
#print('BOTTOM 10 Features')
#print(pd.Series(gb.feature_importances_,index=X.columns).sort_values(ascending=False)[-10:])

TOP 50 Features
Lat                           0.800385
Long                          0.046990
Pop_rad_15km                  0.030238
Population                    0.021378
4$_Restaurant                 0.009710
4$_Restaurant_Pro_capite_2    0.008997
AVG_Altitude                  0.004993
Pizza_V                       0.003676
Vegetarian / Vegan_V          0.003436
EV Charging_V                 0.003347
Gas Station_V                 0.003242
Factory_V                     0.002666
Bookstore_V                   0.002526
Juice Bar_V                   0.002504
Bus Stop_V                    0.002308
Spiritual_V                   0.001929
Men's Store_V                 0.001791
Playground_V                  0.001782
Beach_V                       0.001736
Gift Shop_V                   0.001489
Soccer Field_V                0.001469
Tot_Venues                    0.001431
Café_V                        0.001409
Field_V                       0.001383
Department Store_V            0.001382
Landmark_

In [46]:
gb = GradientBoostingRegressor(random_state=0)
y=df['Taxable_Income_Pro_capite']
X = df.drop(['Lat','Long','Town','Taxable Income','Taxable_Income_Pro_capite', 'Prov', 'Regione'],axis=1)
cv_scores = cross_val_score(gb, X, y,cv=10)
print('All Variables (ex Lat, Long) Mean cross-validation score (10-fold): {:.3f}'.format(np.mean(cv_scores)))

gb = GradientBoostingRegressor(random_state=0)
X = df.drop(['Lat','Long','Town','Taxable Income','Taxable_Income_Pro_capite', 'Prov', 'Regione',],axis=1)
X = X.drop(list(imp[100:].index),axis=1)
cv_scores = cross_val_score(gb, X, y,cv=10)
print('Top 100 Variables (ex Lat, Long) Mean cross-validation score (10-fold): {:.3f}'.format(np.mean(cv_scores)))

gb = GradientBoostingRegressor(random_state=0)
X = df.drop(['Lat','Long','Town','Taxable Income','Taxable_Income_Pro_capite', 'Prov', 'Regione',],axis=1)
X = X.drop(list(imp[300:].index),axis=1)
cv_scores = cross_val_score(gb, X, y,cv=10)
print('Top 300 Variables (ex Lat, Long) Mean cross-validation score (10-fold): {:.3f}'.format(np.mean(cv_scores)))


All Variables (ex Lat, Long) Mean cross-validation score (10-fold): 0.652
Top 100 Variables (ex Lat, Long) Mean cross-validation score (10-fold): 0.596
Top 300 Variables (ex Lat, Long) Mean cross-validation score (10-fold): 0.639


In [64]:
from sklearn.ensemble import AdaBoostRegressor


ada = AdaBoostRegressor(random_state=0)
y=df['Taxable_Income_Pro_capite']
X = df[['Lat','Long']]
cv_scores = cross_val_score(gb, X, y,cv=10)
print('Only Lat and Lng - Mean cross-validation score (10-fold): {:.3f}'.format(np.mean(cv_scores)))


ada = AdaBoostRegressor(random_state=0)
X = df[['Population','Lat','Long','4$_Restaurant','4$_Restaurant_Pro_capite','Pop_rad_15km','Tot_Venues','AVG_Altitude',
        'STD_Altitude','4$_Restaurant_Pro_capite_2']]
cv_scores = cross_val_score(ada, X, y,cv=10)
print('All Macro Variables - Mean cross-validation score (10-fold): {:.3f}'.format(np.mean(cv_scores)))


ada = AdaBoostRegressor(random_state=0)
X = df[['Population','4$_Restaurant','4$_Restaurant_Pro_capite','Pop_rad_15km','Tot_Venues','AVG_Altitude',
        'STD_Altitude','4$_Restaurant_Pro_capite_2']]
cv_scores = cross_val_score(ada, X, y,cv=10)
print('All Macro Variables (ex Lat, Long) - Mean cross-validation score (10-fold): {:.3f}'.format(np.mean(cv_scores)))

ada = AdaBoostRegressor(random_state=0)
y=df['Taxable_Income_Pro_capite']
X = df.drop(['Town','Taxable Income','Taxable_Income_Pro_capite', 'Prov', 'Regione'],axis=1)
cv_scores = cross_val_score(ada, X, y,cv=10)
print('All Variables (inc Lat, Long) Mean cross-validation score (10-fold): {:.3f}'.format(np.mean(cv_scores)))

Only Lat and Lng - Mean cross-validation score (10-fold): 0.809
All Macro Variables - Mean cross-validation score (10-fold): 0.775
All Macro Variables (ex Lat, Long) - Mean cross-validation score (10-fold): 0.469
All Variables (inc Lat, Long) Mean cross-validation score (10-fold): 0.783


In [51]:
X = df.drop(['Town','Taxable Income','Taxable_Income_Pro_capite', 'Prov', 'Regione'],axis=1)
y=df['Taxable_Income_Pro_capite']
ada.fit(X,y)
imp =pd.Series(ada.feature_importances_,index=X.columns)

print('TOP 20 Features')
print(pd.Series(ada.feature_importances_,index=X.columns).sort_values(ascending=False)[:20])
#print('BOTTOM 10 Features')
#print(pd.Series(gb.feature_importances_,index=X.columns).sort_values(ascending=False)[-10:])

TOP 20 Features
Lat                           0.480872
Long                          0.150748
EV Charging_V                 0.031655
Population                    0.022179
Pop_rad_15km                  0.021384
Coworking Space_V             0.018090
Pizza_V                       0.016178
4$_Restaurant                 0.015797
4$_Restaurant_Pro_capite_2    0.013601
Spiritual_V                   0.012302
Steakhouse_V                  0.011652
AVG_Altitude                  0.010825
Government_V                  0.010740
Sporting Goods_V              0.009958
Vegetarian / Vegan_V          0.009425
Coffee Shop_V                 0.007650
Residential_V                 0.007409
Non-Profit_V                  0.007215
Sports Club_V                 0.007190
Harbor / Marina_V             0.006588
dtype: float64


In [52]:
ada = AdaBoostRegressor(random_state=0)
y=df['Taxable_Income_Pro_capite']
X = df.drop(['Lat','Long','Town','Taxable Income','Taxable_Income_Pro_capite', 'Prov', 'Regione'],axis=1)
cv_scores = cross_val_score(ada, X, y,cv=10)
print('All Variables (ex Lat, Long) Mean cross-validation score (10-fold): {:.3f}'.format(np.mean(cv_scores)))

ada = AdaBoostRegressor(random_state=0)
X = df.drop(['Lat','Long','Town','Taxable Income','Taxable_Income_Pro_capite', 'Prov', 'Regione',],axis=1)
X = X.drop(list(imp[100:].index),axis=1)
cv_scores = cross_val_score(ada, X, y,cv=10)
print('Top 100 Variables (ex Lat, Long) Mean cross-validation score (10-fold): {:.3f}'.format(np.mean(cv_scores)))

ada = AdaBoostRegressor(random_state=0)
X = df.drop(['Lat','Long','Town','Taxable Income','Taxable_Income_Pro_capite', 'Prov', 'Regione',],axis=1)
X = X.drop(list(imp[300:].index),axis=1)
cv_scores = cross_val_score(ada, X, y,cv=10)
print('Top 300 Variables (ex Lat, Long) Mean cross-validation score (10-fold): {:.3f}'.format(np.mean(cv_scores)))

All Variables (ex Lat, Long) Mean cross-validation score (10-fold): 0.550
Top 100 Variables (ex Lat, Long) Mean cross-validation score (10-fold): 0.525
Top 300 Variables (ex Lat, Long) Mean cross-validation score (10-fold): 0.553


In [63]:
from sklearn.ensemble import BaggingRegressor
hgb = BaggingRegressor()
y=df['Taxable_Income_Pro_capite']
X = df[['Lat','Long']]
cv_scores = cross_val_score(hgb, X, y,cv=10)
print('Only Lat and Lng - Mean cross-validation score (10-fold): {:.3f}'.format(np.mean(cv_scores)))

hgb = BaggingRegressor()
y=df['Taxable_Income_Pro_capite']
X = df.drop(['Lat','Long','Town','Taxable Income','Taxable_Income_Pro_capite', 'Prov', 'Regione'],axis=1)
cv_scores = cross_val_score(hgb, X, y,cv=10)
print('All Variables (ex Lat, Long) Mean cross-validation score (10-fold): {:.3f}'.format(np.mean(cv_scores)))

hgb = BaggingRegressor()
y=df['Taxable_Income_Pro_capite']
X = df.drop(['Town','Taxable Income','Taxable_Income_Pro_capite', 'Prov', 'Regione'],axis=1)
cv_scores = cross_val_score(hgb, X, y,cv=10)
print('All Variables (incl. Lat, Long) Mean cross-validation score (10-fold): {:.3f}'.format(np.mean(cv_scores)))

Only Lat and Lng - Mean cross-validation score (10-fold): 0.807
All Variables (ex Lat, Long) Mean cross-validation score (10-fold): 0.561
All Variables (incl. Lat, Long) Mean cross-validation score (10-fold): 0.822


In [68]:
import xgboost as xgb
xgb_model  = xgb.XGBRegressor(objective="reg:squarederror", random_state=42)
y=df['Taxable_Income_Pro_capite']
X = df[['Lat','Long']]
cv_scores = cross_val_score(xgb_model, X, y,cv=10)
print('Only Lat and Lng - Mean cross-validation score (10-fold): {:.3f}'.format(np.mean(cv_scores)))

xgb_model  = xgb.XGBRegressor(objective="reg:squarederror", random_state=42)
y=df['Taxable_Income_Pro_capite']
X = df.drop(['Lat','Long','Town','Taxable Income','Taxable_Income_Pro_capite', 'Prov', 'Regione'],axis=1)
cv_scores = cross_val_score(xgb_model, X, y,cv=10)
print('All Variables (ex Lat, Long) Mean cross-validation score (10-fold): {:.3f}'.format(np.mean(cv_scores)))

xgb_model  = xgb.XGBRegressor(objective="reg:squarederror", random_state=42)
y=df['Taxable_Income_Pro_capite']
X = df.drop(['Town','Taxable Income','Taxable_Income_Pro_capite', 'Prov', 'Regione'],axis=1)
cv_scores = cross_val_score(xgb_model, X, y,cv=10)
print('All Variables (incl. Lat, Long) Mean cross-validation score (10-fold): {:.3f}'.format(np.mean(cv_scores)))


Only Lat and Lng - Mean cross-validation score (10-fold): 0.806
All Variables (ex Lat, Long) Mean cross-validation score (10-fold): 0.657
All Variables (incl. Lat, Long) Mean cross-validation score (10-fold): 0.827


In [9]:
import numpy as np

from scipy.stats import uniform, randint

from sklearn.datasets import load_breast_cancer, load_diabetes, load_wine
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split


import xgboost as xgb

X = df.drop(['Town','Taxable Income','Taxable_Income_Pro_capite', 'Prov', 'Regione'],axis=1)
y=df['Taxable_Income_Pro_capite']

def display_scores(scores):
    print("Scores: {0}\nMean: {1:.3f}\nStd: {2:.3f}".format(scores, np.mean(scores), np.std(scores)))

def report_best_scores(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")




xgb_model = xgb.XGBRegressor(objective='reg:squarederror')

params = {
    "colsample_bytree": uniform(0.7, 0.3),
    "gamma": uniform(0, 0.5),
    "learning_rate": uniform(0.03, 0.3), # default 0.1 
    "max_depth": randint(2, 6), # default 3
    "n_estimators": randint(100, 150), # default 100
    "subsample": uniform(0.6, 0.4)
}

search = RandomizedSearchCV(xgb_model, param_distributions=params, random_state=42, n_iter=200, cv=3, verbose=1, n_jobs=1, return_train_score=True)

search.fit(X, y)
report_best_scores(search.cv_results_, 1)



Fitting 3 folds for each of 200 candidates, totalling 600 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 600 out of 600 | elapsed: 12.2min finished


Model with rank: 1
Mean validation score: 0.819 (std: 0.035)
Parameters: {'colsample_bytree': 0.9266653415629146, 'gamma': 0.11439908274581123, 'learning_rate': 0.053093972948637894, 'max_depth': 4, 'n_estimators': 121, 'subsample': 0.9521871356061031}

